# Segmenting and Clustering Neighborhoods in Toronto - Part 1 
(Web scrapping wiki page)

Import the required libraries and read the wiki page into html_doc variable

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
results = requests.get(wiki_url)
#print(results)
html_doc = results.content
#print(html_doc)
print("complete")

complete


Use BeautifulSoup library to identify the class table by name "wikitable sortable" and collect all data into a data array

In [3]:
soup = BeautifulSoup(html_doc, 'html.parser')
#print(soup.prettify())

table = soup.find('table', attrs={'class':'wikitable sortable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
data = []

for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty values
print(data[:5])

[[], ['M1A', 'Not assigned', 'Not assigned'], ['M2A', 'Not assigned', 'Not assigned'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village']]




Clean data to remove Borough's which are Not Assigned and populate details for Neighbourhood which are Not Assigned. Assing the clean data to a new array data_new



In [4]:
data_new = []
data_new.append(['PostCode','Borough','Neighbourhood'])
for i in range(0,len(data)):
    if len(data[i]) == 0:
        continue
    elif data[i][1] == "Not assigned":
        continue
    elif data[i][1] != "Not assigned" and data[i][2] == "Not assigned":
        data[i][2] = data[i][1]
        data_new.append(data[i])   
    else:
        data_new.append(data[i])   
print(data_new[:5])

[['PostCode', 'Borough', 'Neighbourhood'], ['M3A', 'North York', 'Parkwoods'], ['M4A', 'North York', 'Victoria Village'], ['M5A', 'Downtown Toronto', 'Harbourfront'], ['M5A', 'Downtown Toronto', 'Regent Park']]


Create Dataframe from the array created earlier and index based on Postcode to find the unique values.
Once the unique values are identified, concatenate the Neighbourhood details into a single entitiy and prepare a final dataframe

In [15]:

df = pd.DataFrame(data_new[1:],columns=data_new[0])

df.set_index("PostCode", inplace=True)

indx = list(df.index.unique())

final_arr = np.array([['PostCode','Borough','Neighbourhood']])
for j in range(0,len(indx)):
#for j in range(0,3):

    tmp_arr = list(df.loc[[indx[j]],['Neighbourhood']].values)
    
    tmp_str = ""
    for k in range(0,len(tmp_arr)):
        val = tmp_arr[k].tolist()
        
        if k == 0:
            tmp_str = val[0]
        else:
            tmp_str = tmp_str + "," + val[0]

    borough_details = np.unique(list(df.loc[[indx[j]],['Borough']].values))

    borough_str = borough_details[0]

    arr = [indx[j],borough_str,tmp_str]
    final_arr = np.append(final_arr, [arr], axis = 0) 

final_dataset = pd.DataFrame(final_arr[1:], columns=final_arr[0])
final_dataset.head()


,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [6]:
final_dataset.shape

(103, 3)

# Segmenting and Clustering Neighborhoods in Toronto - Part 2

In [7]:
#!wget -q -O 'toronto_geo_data.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


Load the toronto geo data from csv and set index on PostCode column in the required dataframes

In [16]:
geo_data = pd.read_csv('toronto_geo_data.csv')

geo_data = geo_data.rename(columns={'Postal Code': 'PostCode'})
geo_data.set_index('PostCode',inplace=True)
geo_data.head()


,Latitude,Longitude
PostCode,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [17]:
final_dataset.set_index('PostCode',inplace=True)
final_dataset.head()

,Borough,Neighbourhood
PostCode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park


In [25]:
df_2 = final_dataset.join(geo_data)
df_2.reset_index(drop=False, inplace=True)
df_2.head()

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


In [26]:
df_2.shape

(103, 5)